# Case study for data reuse with EBRAINS Knowledge Graph: A step-by-step explanation 

Alix E.Bonard, Laura Morel and Peyman Najafi

Paris-Saclay Institute of Neuroscience, CNRS, Université Paris-Saclay, France.

November 2024, NeuralNet2024 - Minischool - Hands-on case studies for data reuse 


In this notebook we will look more closely at the EBRAINS dataset "[Excitability profile of CA1 pyramidal neurons in APPPS1 Alzheimer disease mice and control littermates (v1)](https://search.kg.ebrains.eu/#bd5f91ff-e829-4b85-92eb-fc56991541f1)", contributed by Ana Rita Salgueiro-Pereira and Hélène Marie from the Université Côte d’Azur in Valbonne, France.

As we can see from the dataset description,

<i>This dataset provides an analysis of the intrinsic electrophysiological properties of CA1 excitatory hippocampal
neurons in a mouse model of Alzheimer’s Disease (AD) at two age points: a presymptomatic age (3-4
months) and a symptomatic age: (9-10 months).</i>
More information is available in the [Data Descriptor](https://search.kg.ebrains.eu/instances/bd5f91ff-e829-4b85-92eb-fc56991541f1).

This dataset forms part of the results reported in Vitale, P., Salgueiro-Pereira, A. R., Lupascu, C. A., Willem, M., Migliore, R., Migliore, M., & Marie, H.(2021) Analysis of Age-Dependent Alterations in Excitability Properties of CA1 Pyramidal Neurons in an APPPS1 Model of Alzheimer’s Disease. *Frontiers in Aging Neuroscience* **13** https://doi.org/10.3389/fnagi.2021.668948

This notebook was modified from [Studies of data reuse: Excitability profile of CA1 pyramidal neurons in APPPS1 Alzheimer disease mice and control littermates](https://github.com/NeuroPSI-Neuroinformatics/case-studies-for-data-reuse/blob/main/10.25493-YJFW-HPY/SalgueiroPereiraMarie2020.ipynb) by Isaure Botherel and Andrew P.Davison. 


**In this notebook we will demonstrate how to access, to explore and to reuse the data from this study using fairgraph and neo packages. We will also visualize the data with matplotlib by focusing on providing detailed information for data reuse.**

### Finding a dataset

This section aims to teach users how to programatically or manually find and retrieve datasets with specific metadata on the EBRAINS Knowledge Graph (KG). The following tools will be used:
- the python library fairgraph
- the EBRAINS website KGSearch
- the EBRAINS website KGQueryBuilder

#### Using [KGSearch](https://search.kg.ebrains.eu/?category=Dataset)

[KGSearch](https://search.kg.ebrains.eu/?category=Dataset) is a quick and easy way to browse EBRAINS datasets. Once a dataset is selected, users can use the Python library [fairgraph](https://github.com/HumanBrainProject/fairgraph) to download the dataset and/or see its metadata using its client. Since we are already working in a Jupyter lab linked to our EBRAINS account, we simply need to create a client object **KGClient** to be able to retrieve metadata on the KG. For more instructions on how to use fairgraph in other environments, please refer to the github page linked above.

In [ ]:
from fairgraph import KGClient
kg_client = KGClient()

**EXERCISE**\
To familiarize ourselves with KGSearch, let us try to find a dataset with the following characteristics:
- Technique used is **whole-cell patch clamp**
- Cell type is **CA1 pyramidal neurons**
- Sujects are **model mice for Alzheimer's disease**

Those parameters should lead you to [this dataset](https://search.kg.ebrains.eu/instances/bd5f91ff-e829-4b85-92eb-fc56991541f1)

The KG uses the [openMinds](https://github.com/openMetadataInitiative/openMINDS) framework for metadata. In this framework, a **Dataset** instance is linked to one or more **DatasetVersion** instances with unique DOIs. *fairgraph* allows us to directly download the dataset using its DOI.

In [ ]:
import fairgraph.openminds.core as omcore
import os 
import zipfile

dataset_version_doi = "10.25493/YJFW-HPY"

dataset_versions = omcore.DatasetVersion.list(
    kg_client, 
    digital_identifier__identifier=dataset_version_doi,
    follow_links={"repository": {"files": {}}},
    scope="any"
)

dataset_version = dataset_versions[0]

if not os.path.exists("downloads"):  # only download the dataset if it hasn't been downloaded previously
    dataset_path = dataset_version.download("downloads", kg_client, accept_terms_of_use=True)[0]
else:
    dataset_path = "downloads/ext-d000001_ADNeuronModel_pub"
with zipfile.ZipFile(dataset_path, "r") as z:
    z.extractall("downloads")

#### More sophisticated queries with [KGQueryBuilder](https://query.kg.ebrains.eu)

To query the KG programatically, instead of using the KGSearch website, we can use the KGQuery object from fairgraph. However, the query must be serializable in JSON-LD to be valid. The [KGQueryBuilder](https://query.kg.ebrains.eu) website aims to help users write queries for the EBRAINS Knowledge Graph.
Using this tool, try and find the dataset previously found. [A tutorial](https://docs.kg.ebrains.eu/9b511d36d7608eafc94ea43c918f16b6/tutorials.html) is available to learn how to use this tool. Once you think you have the correct query, paste it below to check if it is correct.
**Note:** if your query contains the words `true` or `false` without quotes, remember to capitalize the first letter to make it interpretable by Python

**EXERCISE: Build a query with KG Query builder**

In [ ]:
from pprint import pprint
query = {
    # PASTE YOUR QUERY HERE
}
results = kg_client.query(query)
pprint(results.data, width=200)

## Structure of Folder 

The data are organized into four folders, "APPPS1_mouse_model_3-4_months", "control_3-4_months", "APPPS1_mouse_model_9-10_months", "control_9-10_months", each of which contains a number of files with the extension ".abf".

It should be noted that the dataset does not contain the data for mice at age 1 month that are shown in the associated paper (Vitale et al., 2021).

We know from the dataset metadata that these are electrophysiology data, and more specifically that they were obtained with the whole cell patch clamp technique in current clamp mode. We will therefore use the [Neo library](https://neo.readthedocs.io/) to read the data files, since it is able to read data from a large number of electrophysiology file formats.

In [ ]:
data_folder = os.path.abspath(os.listdir()[1]) # gets the path of the second element in the working directory. In this case it is "downloads".  
data_folder

In [ ]:
import os

path_folders_list = []
sub_folders_list = []
files_list = []
for path_folders, sub_folders, files in os.walk(data_folder):
    path_folders_list.append(path_folders)
    sub_folders_list.append(sub_folders)
    files_list.append(files)

print(f"The dataset contains {len(path_folders_list[1:])} folders corresponding to the experimental groups")
for i in range (1, (len(path_folders_list))):
    print(f"The folfer {path_folders_list[i]} contains {len(files_list[i])} files with the extension .abf ({files_list[i]})")


## Exploring Neo package: FAIR and flexible tool to analyze/visualize your data 
FAIR principle related to Neo:\
I1: (meta)data use a formal, accessible, shared, and broadly applicable language for knowledge representation.\
I3: (meta)data include qualified references to other (meta)data\
R1.3: (meta)data meet domain-relevant community standards.


In [1]:
from glob import glob
import numpy as np
from neo import get_io

Neo loads data into a hierarchical structure: Blocks contain Segments, which contain the actual data.



![title](Neo_architecture.png)

Let us look at the structure of the first three files:

In [2]:
for file_path in glob("downloads/*/*.abf")[:3]:
    io = get_io(file_path)
    print(io)

AxonIO: downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf
nb_block: 1
nb_segment:  [13]
signal_streams: [Signals (chans: 1)]
signal_channels: [IN0]
spike_channels: []
event_channels: [Tag]

AxonIO: downloads/APPPS1_mouse_model_3-4_months/191129002_S23.abf
nb_block: 1
nb_segment:  [13]
signal_streams: [Signals (chans: 1)]
signal_channels: [IN0]
spike_channels: []
event_channels: [Tag]

AxonIO: downloads/APPPS1_mouse_model_3-4_months/191129003_S23.abf
nb_block: 1
nb_segment:  [13]
signal_streams: [Signals (chans: 1)]
signal_channels: [IN0]
spike_channels: []
event_channels: [Tag]



**EXERCISE: Knowing the structure of Neo object, retrieve the signal ?**

Hint:

block = get_io("downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf").read(lazy=True) #using read(lazy=True) implies to use .load() to retrieve signals\
...

--\
Output:

AnalogSignal with 1 channels of length 50000; units mV; datatype float32\
name: 'Signals'\
annotations: {'stream_id': '0'}\
sampling rate: 10000.0 Hz\
time: 0.020999999999999998 s to 5.021 s

In [ ]:
# Solution 
data = get_io("downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf").read(lazy=True)
example_signal = data[0].segments[0].analogsignals[0].load()
example_signal

**EXERCISE: Explore the proprieties of the object in order to obtain the raw data, the time (with and without units), the units and the sampling rate**

In [ ]:
# Solution
print(f"Raw signal:  {example_signal.magnitude.flatten()}")
print(f"time with unit:  {example_signal.times}")
print(f"time without unit:  {example_signal.times.magnitude}")
print(f"sampling rate:  {example_signal.sampling_rate}")
print(f"sampling rate without unit:  {example_signal.sampling_rate.magnitude}")
print(f"time unit:  {example_signal.times.dimensionality}")
print(f"signal unit:  {example_signal.units.dimensionality}")

These all have a consistent structure: they're in Axon format, contain a single block containing 13 segments, and each segment contains a single recorded signal containing a single channel. 
We know from the associated paper that current pulses of increasing intensity were injected into the neurons, in steps of 50 pA from -200 to 400 pA.

In [4]:
current_pulse_amplitudes = np.arange(-200, 401, 50)  # we set the upper limit above 400, so that the final value is 400
print(current_pulse_amplitudes)
print(f"Number of current pulses: {len(current_pulse_amplitudes)}")

[-200 -150 -100  -50    0   50  100  150  200  250  300  350  400]
Number of current pulses: 13


Now let's plot the data from one of the files:

**EXERCISE: Plot the membrane potential(signal) over the time for all segments**

In [ ]:
# Solution
import matplotlib.pyplot as plt

def plot_data(file_path):
    data = get_io(file_path).read(lazy=True)
    for segment in data[0].segments:
        signal = segment.analogsignals[0].load()
        plt.plot(signal.times, signal)
    plt.xlabel(f"Time ({signal.times.units.dimensionality})")
    plt.ylabel(f"Membrane potential ({signal.units.dimensionality})")

plot_data("downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf")

![title](Fig1.png)

**EXERCISE: To get a closer look at the signals, (1) shift the time axis to be relative to the start time of each signal, and (2) plot only the 500 ms around the current injection. Adjust the code used above to do so.**

HINT: signal.times - signal.t_start

In [ ]:
# Solution
def plot_data_zoom(file_path):
    data = get_io(file_path).read(lazy=True)
    for segment in data[0].segments:
        signal = segment.analogsignals[0].load()
        plt.plot(signal.times - signal.t_start, signal)
    plt.xlim(0.05, 0.55)
    plt.xlabel(f"Time ({signal.times.units.dimensionality})")
    plt.ylabel(f"Membrane potential ({signal.units.dimensionality})")

plot_data_zoom("downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf")

![title](Fig2.png)

**EXERCISE: Restrict the signal to 1 second? to 0.6 second?**

Hint: np.where()

In [ ]:
def plot_data_vs(file_path, value_second):
    data = get_io(file_path).read(lazy=True)
    for segment in data[0].segments:
        signal = segment.analogsignals[0].load()
        indices_vs = np.where((signal.times - signal.t_start) <= value_second)[0]
        plt.plot(signal.times[indices_vs]-signal.t_start, signal.magnitude[indices_vs])
    
    plt.xlabel(f"Time ({signal.times.units.dimensionality})")
    plt.ylabel(f"Membrane potential ({signal.units.dimensionality})")

plot_data_vs("downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf", 0.6)

### Data reuse with matplotlib

**EXERCISE: Complete the code below in order to plot the membrane potential over the current amplitudes?**

In [ ]:
mean_membrane_potential = []
value_second = 0.6

data = get_io('downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf').read(lazy=True)
for segment in data[0].segments:
    signal = segment.analogsignals[0].load()
    indices_1s = np.where((signal.times - signal.t_start) <= value_second)[0]

    mean_membrane_potential.append(np.mean(signal.magnitude[indices_1s]))
    
colors = plt.cm.tab20(np.linspace(0, 1, 13))
plt.scatter(current_pulse_amplitudes,mean_membrane_potential, color= colors, zorder= 2)
plt.plot(current_pulse_amplitudes,mean_membrane_potential,'-', color = 'k', alpha = 0.5, zorder=1)
plt.xlabel(f"Currents (pA)", size = 15)
plt.ylabel(f"Mean Membrane potential ({signal.units.dimensionality})", size = 15)

ax = plt.gca()

ax.set_xlim(-250,450)
ax.set_xticks([-200,-100,0,100,200,300,400,450])
ax.set_xticklabels(['-200','-100','0','100','200','300','400', ' '], size = 12)
ax.tick_params(axis='x', which='major', size=5) 
ax.set_xticks([-250,-150,50,-50,150,250,350], minor=True) 
ax.tick_params(axis='x', which='minor', size=3) 

ax.set_ylim(-85,-40)
ax.set_yticks([-80,-70,-60,-50,-40])
ax.set_yticklabels(['-80','-70','-60','-50','-40'], size = 12)
ax.tick_params(axis='y', which='major', size=5) 
ax.set_yticks([-85,-75,-65,-55,-45], minor=True) 
ax.tick_params(axis='y', which='minor', size=3) 

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.show()

**EXERCISE: Reproduce this figure**

HINT:

from matplotlib.ticker import MultipleLocator

for i in range(len(current_pulse_amplitudes)):
    plt.annotate(f'{annotation_points[i]}', 
                 (current_pulse_amplitudes[i], mean_membrane_potential[i]),        
                 textcoords="offset points", 
                 xytext=(5,-10),       
                 ha='left')        

ax = gca()

ax.set_xlim(-250,450)
ax.set_xticks([-200,-100,0,100,200,300,400,450])
ax.set_xticklabels(['-200','-100','0','100','200','300','400', ' '], size = 12)
ax.tick_params(axis='...', which='...', size=5) 
ax.set_xticks([-250,-150,50,-50,150,250,350], minor=True) 
ax.tick_params(axis='...', which='...', size=3) 

ax.set_ylim(-85,-40)
ax.set_yticks([-80,-70,-60,-50,-40])
ax.set_yticklabels(['-80','-70','-60','-50','-40'], size = 12)
ax.set_yticks([-85,-75,-65,-55,-45], minor=True) 
ax.yaxis.set_minor_locator(...(1))   
ax.yaxis.set_major_locator(...(5))   
ax.tick_params(axis='...', which='minor', size=3) 
ax.tick_params(axis='...', which='major', size=5) 

ax.spines['...'].set_visible(False)
ax.spines['...'].set_visible(False)

plt.show()

![title](Figmatplotlib.png) 


In [ ]:
from matplotlib.ticker import MultipleLocator


mean_membrane_potential = []
value_second = 0.6

data = get_io('downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf').read(lazy=True)
for segment in data[0].segments:
    signal = segment.analogsignals[0].load()
    indices_1s = np.where((signal.times - signal.t_start) <= value_second)[0]

    mean_membrane_potential.append(np.mean(signal.magnitude[indices_1s]))

# Plot & layout  
# 
# Main Graph  
colors = plt.cm.tab20(np.linspace(0, 1, 13))
plt.scatter(current_pulse_amplitudes,mean_membrane_potential, color= colors, zorder= 2)
plt.plot(current_pulse_amplitudes,mean_membrane_potential,'-', color = 'k', alpha = 0.5, zorder=1)
plt.xlabel(f"Currents (pA)", size = 15)
plt.ylabel(f"Mean Membrane potential ({signal.units.dimensionality})", size = 15)

# Graph Annotations
annotation_points = ['point1','point2','point3','point4',' ',' ',' ',' ',' ',' ',' ','point12','point13']
for i in range(len(current_pulse_amplitudes)):
    plt.annotate(f'{annotation_points[i]}', 
                 (current_pulse_amplitudes[i], mean_membrane_potential[i]),        
                 textcoords="offset points", 
                 xytext=(5,-10),       
                 ha='left')        


ax = plt.gca()

ax.set_xlim(-250,450)
ax.set_xticks([-200,-100,0,100,200,300,400,450])
ax.set_xticklabels(['-200','-100','0','100','200','300','400', ' '], size = 12)
ax.tick_params(axis='x', which='major', size=5) 
ax.set_xticks([-250,-150,50,-50,150,250,350], minor=True) 
ax.tick_params(axis='x', which='minor', size=3) 

ax.set_ylim(-85,-40)
ax.set_yticks([-80,-70,-60,-50,-40])
ax.set_yticklabels(['-80','-70','-60','-50','-40'], size = 12)
ax.set_yticks([-85,-75,-65,-55,-45], minor=True) 
ax.yaxis.set_minor_locator(MultipleLocator(1))   
ax.yaxis.set_major_locator(MultipleLocator(5))   
ax.tick_params(axis='y', which='minor', size=3) 
ax.tick_params(axis='y', which='major', size=5) 

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

**EXERCISE: Create subplots next to the points**

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

mean_membrane_potential = []
value_second = 0.6

data = get_io('downloads/APPPS1_mouse_model_3-4_months/191129000_S23.abf').read(lazy=True)
for segment in data[0].segments:
    signal = segment.analogsignals[0].load()
    indices_1s = np.where((signal.times - signal.t_start) <= value_second)[0]

    mean_membrane_potential.append(np.mean(signal.magnitude[indices_1s]))

# Plot & layout  
# 
fig, ax = plt.subplots(figsize=(18, 12))  

# Main Graph  
colors = plt.cm.tab20(np.linspace(0, 1, 13))
plt.scatter(current_pulse_amplitudes,mean_membrane_potential, color= colors, zorder= 2)
plt.plot(current_pulse_amplitudes,mean_membrane_potential,'-', color = 'k', alpha = 0.5, zorder=1)
plt.xlabel(f"Currents (pA)", size = 15)
plt.ylabel(f"Mean Membrane potential ({signal.units.dimensionality})", size = 15)

# Graph Annotations
annotation_points = ['point1','point2','point3','point4',' ',' ',' ',' ',' ',' ',' ','point12','point13']
for i in range(len(current_pulse_amplitudes)):
    plt.annotate(f'{annotation_points[i]}', 
                 (current_pulse_amplitudes[i], mean_membrane_potential[i]),        
                 textcoords="offset points", 
                 xytext=(-15,2),       
                 ha='right',
                 size=15)        

# Subplots  
for i, segment in enumerate(data[0].segments):
    signal = segment.analogsignals[0].load()
    indices_vs = np.where((signal.times - signal.t_start) <= value_second)[0]

    signal_vs = signal.magnitude[indices_vs]
    time_vs = signal.times[indices_vs]

    # creates boxes next to each point
    inset = inset_axes(ax,
                       width="200%", height="100%",
                       loc="...",
                       bbox_to_anchor=(current_pulse_amplitudes[i], mean_membrane_potential[i], 20, 3),  # Anchor the box
                       bbox_transform=ax.transData, 
                       borderpad=5)
    
    # inserts plots in boxes
    inset.plot(...,  color=colors[i])
    
    # Remove spines of all boxes
    for spine in inset.spines.values():
        spine.set...
    
    # Remove ticks for all boxes
    inset.set_xticks([])

# Move the main axes
ax.spines['bottom']...(('data',-87))
ax.spines['left']....(('data',-250))

#
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xlim(-250,450)
ax.set_xticks([-200,-100,0,100,200,300,400,450])
ax.set_xticklabels(['-200','-100','0','100','200','300','400', ' '], size = 12)
ax.tick_params(axis='x', which='major', size=7) 
ax.set_xticks([-250,-150,50,-50,150,250,350], minor=True) 
ax.tick_params(axis='x', which='minor', size=5) 

ax.set_ylim(-87,-40)
ax.set_yticks([-80,-70,-60,-50,-40])
ax.set_yticklabels(['-80','-70','-60','-50','-40'], size = 12)
ax.tick_params(axis='y', which='major', size=7) 
ax.set_yticks([-85,-75,-65,-55,-45], minor=True) 
ax.tick_params(axis='y', which='minor', size=5) 
ax.yaxis.set_minor_locator(MultipleLocator(1))   
ax.yaxis.set_major_locator(MultipleLocator(5))   

# Comments: don't forget to put the legend with plt.legend() : https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.legend.html

plt.show()

### Creating a Neo object

For this part, we want to create an Neo object from hypothetical data. These data can be find in the Minischool_NeuralNet2024_data_example.xlsx file. 

We use pandas package to acces and visualize the data. 

In [ ]:
import pandas as pd
df_data = pd.read_excel("Minischool_NeuralNet2024_data_example.xlsx")
df_data

To do so, we create a function to convert the pandas columns into numpy arrays. The function also store the array in a dictionnary to facilitate the accessibility. 

In [ ]:
def pandas_to_numpy(df):
    arrays = {}
    for col in df.columns:
        arrays[col] = df[col].to_numpy()
    return arrays


Let's have a look: 

In [ ]:
data = pandas_to_numpy(df_data)
data.keys()

We will now create the Neo object according to the structure explained above:\
We start with formating the signals in AnalogSignals

In [163]:
from neo.core import AnalogSignal,Segment,Block
import quantities as pq

sig0 = AnalogSignal(data['Signal 1'], units='V', sampling_rate=1000*pq.Hz, t_start=0*pq.s)
sig1 = AnalogSignal(data['Signal 2'], units='V', sampling_rate=1000*pq.Hz, t_start=0*pq.s)
sig2 = AnalogSignal(data['Signal 3'], units='V', sampling_rate=1000*pq.Hz, t_start=0*pq.s)
sig3 = AnalogSignal(data['Signal 4'], units='V', sampling_rate=1000*pq.Hz, t_start=0*pq.s)
sig4 = AnalogSignal(data['Signal 5'], units='V', sampling_rate=1000*pq.Hz, t_start=0*pq.s)
    

Now, we create the segment regrouping all signals 

In [168]:
seg = Segment(index=5)
seg.analogsignals.append(sig0)
seg.analogsignals.append(sig1)
seg.analogsignals.append(sig2)
seg.analogsignals.append(sig3)
seg.analogsignals.append(sig4)

Let's have a look:

And then, we create the block containing 1 segment and 5 signals(analogsignals)

In [ ]:
blk = Block()
for ind in range(3):
        blk.segments.append(seg)
blk.annotate(description = "data for minischool")
blk

To go further, you can check this example: https://neo.readthedocs.io/en/latest/share_data.html